In [1]:
import os
import cv2

import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import pandas as pd

from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import to_categorical
from keras import regularizers
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage import exposure
from random import sample

import wandb
from wandb import AlertLevel
from datetime import timedelta

wandb.login()

wandb: Currently logged in as: camperko (use `wandb login --relogin` to force relogin)


True

In [7]:
labels = pd.read_csv('data/train.csv')
labels.head()

'scab frog_eye_leaf_spot complex'

In [10]:
categories = []
for index, row in labels.iterrows():
    parts = row['labels'].split(' ')
    for part in parts:
        if part not in categories:
            categories.append(part)

for category in categories:
    labels[category] = np.zeros(labels.shape[0])

for index, row in labels.iterrows():
    parts = row['labels'].split(' ')
    for part in parts:
        labels[part][index] = 1

labels = labels.drop(['labels'], axis=1)

<ipython-input-10-7c883e46b122>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels[part][index] = 1


In [ ]:
config = {"lr": 1e-5,
          "num_epoch": 100,
          "size": 512,
          "optimizer": "adam"}

def normalize_image(in_image):
    sharpen_kernel = np.array([[-0.5, -0.5, -0.5],
                               [-0.5,    5, -0.5],
                               [-0.5, -0.5, -0.5]])

    # resized
    resized_image = resize(in_image, (config["size"], config["size"]), anti_aliasing=True)

    # with expo
    # expo_image = exposure.equalize_adapthist(resized_image, clip_limit=0.03)

    # after sharpen expo
    sharpen_image = cv2.filter2D(resized_image, -1, sharpen_kernel)

    # after gray sharpen expo
    # sharpen_expo_gray_image = rgb2gray(sharpen_expo_image)

    return sharpen_image

image_set = []

for image in labels['image']:
    image_normalized = normalize_image(imread('data/train_images/' + image))
    image_set.append(image_normalized)

In [ ]:
model = Sequential()

model.add(Conv2D(64, input_shape=(100, 100, 3), kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(26, activation='softmax'))

model.summary()
